# Polymarket best-price duration analysis

This notebook studies **how long the best prices (best bid / best ask) remain unchanged** in Polymarket order books. We focus on per-market behavior with a deep dive into a selected market, plus optional aggregates across multiple markets for broader context.

**Goal:** quantify the *persistence* of top-of-book prices by measuring the duration of each run where the best bid or best ask stays constant.


In [ ]:
import os
import sys
import subprocess
import importlib.util
from pathlib import Path

# Install lightweight dependencies once per environment

def ensure_package(pkg_name: str) -> None:
    if importlib.util.find_spec(pkg_name) is None:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg_name])

for pkg in ("pandas", "sqlalchemy", "python-dotenv", "psycopg2-binary", "plotly", "numpy"):
    ensure_package(pkg)

import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import plotly.express as px
import plotly.graph_objects as go


In [ ]:
# Load DATABASE_URL from repo .env if present
root = Path.cwd()
load_dotenv(root / ".env")

DATABASE_URL = os.getenv("DATABASE_URL")
if DATABASE_URL and DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)
if not DATABASE_URL:
    raise RuntimeError("DATABASE_URL not set. Add to .env or environment.")

engine = create_engine(DATABASE_URL)


## 1) Market overview

We start by finding markets with the most orderbook events. Pick a `market_instance_id` to analyze in depth.


In [ ]:
market_overview_query = """
SELECT
    market_instance_id,
    market,
    asset_label,
    COUNT(*) AS num_events,
    MIN(event_timestamp_ms) AS min_ts,
    MAX(event_timestamp_ms) AS max_ts
FROM polymarket_orderbook_events
GROUP BY market_instance_id, market, asset_label
ORDER BY num_events DESC
LIMIT 25;
"""

market_overview = pd.read_sql(market_overview_query, engine)
market_overview["span_minutes"] = (market_overview["max_ts"] - market_overview["min_ts"]) / 60000.0
market_overview


## 2) Select a market to analyze

Choose a `market_instance_id` from the table above. This analysis is designed to be deep on one market to understand best-price persistence in detail.


In [ ]:
# Pick the market with the most events by default
selected_market = market_overview.iloc[0]["market_instance_id"]
selected_market


## 3) Load orderbook events for the selected market

We pull best bid/ask prices per event for each side (UP/DOWN).


In [ ]:
events_query = """
SELECT
    event_timestamp_ms,
    side,
    best_bid_price,
    best_ask_price,
    best_bid_qty,
    best_ask_qty
FROM polymarket_orderbook_events
WHERE market_instance_id = %(market)s
ORDER BY event_timestamp_ms ASC;
"""

pm = pd.read_sql(events_query, engine, params={"market": selected_market})
if pm.empty:
    raise RuntimeError("No orderbook data found for selected market.")

pm["event_ts"] = pd.to_datetime(pm["event_timestamp_ms"], unit="ms", utc=True)
pm.head()


## 4) Helper: compute best-price duration runs

We measure how long each *unchanged* price run lasts. For each side, we compute consecutive runs where the best bid or best ask stays the same, and measure the duration until the next price change.


In [ ]:
def price_run_durations(df: pd.DataFrame, price_col: str) -> pd.DataFrame:
    # df must be sorted by event_timestamp_ms
    df = df.copy()
    df = df.sort_values("event_timestamp_ms")

    # Identify changes in price
    price_change = df[price_col].ne(df[price_col].shift(1))
    df["run_id"] = price_change.cumsum()

    # For each run, take the first timestamp and price
    run_summary = (
        df.groupby("run_id")[["event_timestamp_ms", price_col]]
        .first()
        .rename(columns={"event_timestamp_ms": "run_start_ms", price_col: "price"})
        .reset_index(drop=True)
    )

    # Run end is the start of the next run
    run_summary["run_end_ms"] = run_summary["run_start_ms"].shift(-1)
    run_summary["duration_ms"] = run_summary["run_end_ms"] - run_summary["run_start_ms"]

    # Drop final run (no end timestamp)
    run_summary = run_summary.dropna(subset=["duration_ms"])
    run_summary["duration_sec"] = run_summary["duration_ms"] / 1000.0
    return run_summary


## 5) Best bid duration analysis

This tells us how long the **best bid price** stays unchanged before moving.


In [ ]:
bid_runs = []
for side, group in pm.groupby("side"):
    runs = price_run_durations(group, "best_bid_price")
    runs["side"] = side
    runs["price_type"] = "best_bid"
    bid_runs.append(runs)

bid_runs = pd.concat(bid_runs, ignore_index=True)
bid_runs.head()


In [ ]:
bid_summary = (
    bid_runs.groupby("side")["duration_sec"]
    .describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
    .reset_index()
)
bid_summary


In [ ]:
fig = px.histogram(
    bid_runs,
    x="duration_sec",
    color="side",
    nbins=60,
    title="Best Bid Price Persistence (seconds)",
)
fig.update_layout(xaxis_title="Duration (sec)", yaxis_title="Count")
fig.show()


## 6) Best ask duration analysis

This tells us how long the **best ask price** stays unchanged before moving.


In [ ]:
ask_runs = []
for side, group in pm.groupby("side"):
    runs = price_run_durations(group, "best_ask_price")
    runs["side"] = side
    runs["price_type"] = "best_ask"
    ask_runs.append(runs)

ask_runs = pd.concat(ask_runs, ignore_index=True)
ask_runs.head()


In [ ]:
ask_summary = (
    ask_runs.groupby("side")["duration_sec"]
    .describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
    .reset_index()
)
ask_summary


In [ ]:
fig = px.histogram(
    ask_runs,
    x="duration_sec",
    color="side",
    nbins=60,
    title="Best Ask Price Persistence (seconds)",
)
fig.update_layout(xaxis_title="Duration (sec)", yaxis_title="Count")
fig.show()


## 7) Compare bid vs ask persistence

We can compare distributions to see whether bids or asks tend to be more stable.


In [ ]:
combined_runs = pd.concat([bid_runs, ask_runs], ignore_index=True)

fig = px.box(
    combined_runs,
    x="price_type",
    y="duration_sec",
    color="side",
    points="outliers",
    title="Best Price Duration by Side and Type",
)
fig.update_layout(yaxis_title="Duration (sec)")
fig.show()


## 8) Share of time at a given best price

We can estimate how concentrated time is in the most persistent price levels by weighting durations. This reveals whether a few price levels dominate the time spent at top-of-book.


In [ ]:
def top_price_concentration(runs: pd.DataFrame, top_n: int = 5) -> pd.DataFrame:
    grouped = (
        runs.groupby(["side", "price"])["duration_sec"]
        .sum()
        .reset_index()
        .sort_values(["side", "duration_sec"], ascending=[True, False])
    )
    totals = grouped.groupby("side")["duration_sec"].sum().rename("total_sec")
    grouped = grouped.merge(totals, on="side")
    grouped["share"] = grouped["duration_sec"] / grouped["total_sec"]
    return grouped.groupby("side").head(top_n)

concentration_bid = top_price_concentration(bid_runs, top_n=5)
concentration_ask = top_price_concentration(ask_runs, top_n=5)

concentration_bid


In [ ]:
concentration_ask


## 9) Optional: Multi-market aggregate (SQL window approach)

For a broader view, this SQL computes *run durations* in the database using window functions. This can be heavy depending on dataset size, so start with a small limit or filter.


In [ ]:
aggregate_query = """
WITH ordered AS (
    SELECT
        market_instance_id,
        side,
        event_timestamp_ms,
        best_bid_price,
        best_ask_price,
        LAG(best_bid_price) OVER (PARTITION BY market_instance_id, side ORDER BY event_timestamp_ms) AS prev_bid,
        LAG(best_ask_price) OVER (PARTITION BY market_instance_id, side ORDER BY event_timestamp_ms) AS prev_ask
    FROM polymarket_orderbook_events
), runs AS (
    SELECT
        *,
        CASE WHEN best_bid_price = prev_bid THEN 0 ELSE 1 END AS bid_change,
        CASE WHEN best_ask_price = prev_ask THEN 0 ELSE 1 END AS ask_change
    FROM ordered
), bid_runs AS (
    SELECT
        market_instance_id,
        side,
        event_timestamp_ms,
        best_bid_price,
        SUM(bid_change) OVER (PARTITION BY market_instance_id, side ORDER BY event_timestamp_ms) AS bid_run_id
    FROM runs
), bid_run_bounds AS (
    SELECT
        market_instance_id,
        side,
        bid_run_id,
        MIN(event_timestamp_ms) AS run_start_ms
    FROM bid_runs
    GROUP BY market_instance_id, side, bid_run_id
), bid_run_durations AS (
    SELECT
        market_instance_id,
        side,
        run_start_ms,
        LEAD(run_start_ms) OVER (PARTITION BY market_instance_id, side ORDER BY run_start_ms) AS run_end_ms
    FROM bid_run_bounds
)
SELECT
    side,
    COUNT(*) AS num_runs,
    AVG((run_end_ms - run_start_ms) / 1000.0) AS avg_duration_sec,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY (run_end_ms - run_start_ms) / 1000.0) AS median_duration_sec
FROM bid_run_durations
WHERE run_end_ms IS NOT NULL
GROUP BY side;
"""

# Uncomment to run if your database can handle a larger aggregate
# aggregate_summary = pd.read_sql(aggregate_query, engine)
# aggregate_summary


## 10) Interpretation prompts

Use these prompts to guide your analysis and writeup:

- Are best bids more stable than best asks? Does this differ by side (UP vs DOWN)?
- Do a few price levels account for most of the time at top-of-book?
- Are there notable asymmetries between UP and DOWN sides?
- Does price persistence change during high-volatility windows (you can filter by time to test)?

Feel free to extend the notebook with additional filters (time windows, volume thresholds) or to compare multiple markets using the aggregate SQL approach above.
